## Extraction des données de Reddit avec PRAW

### Importation des packages et configuration de l'API

In [ ]:
! pip install praw

In [ ]:
import requests
import praw
import pandas as pd
import time
import json
from datetime import datetime
from tqdm.notebook import tqdm
import logging

# Réduction des logs PRAW
logging.getLogger("praw").setLevel(logging.ERROR)

# ============ CONFIGURATION PRAW ==============
reddit = praw.Reddit(
    client_id="2fnJun-nq4gDKyuRBqojLA",
    client_secret="7NSA8JmbuZXZry-bFOErikBbtAm8tw",
    user_agent="teknoloogiaa"
)

### Scrapping des données

#### Posts

In [ ]:
# ============ SCRAPING DES POSTS ==============

# Fonction de scraping des posts Reddit
def scrapper_posts(liste_theme=["datascience", "Machinelearning"], limit=1000, time_filter="all"):
    posts = reddit.subreddit("+".join(liste_theme)).top(time_filter=time_filter, limit=limit)
    df_posts = []

    for post in tqdm(posts, total=limit, desc="Téléchargement des posts Reddit"):
        df_posts.append({
            "id": post.id,
            "subreddit": post.subreddit.display_name,
            "created_utc": post.created_utc,
            "post_title": post.title,
            "link_flair_text": post.link_flair_text,
        })

    return pd.DataFrame(df_posts)


# Liste des sujets Reddit que nous utiliserons
themes = ["MachineLearning", "ArtificialInteligence", "artificial", "datascience", "deeplearning"]

df_posts = scrapper_posts(liste_theme = themes, limit = 5000, time_filter = "all")
df_posts.to_csv("/content/df_posts.csv", header = True, index = False)
df_posts.head()

#### Commentaires

In [ ]:
# ============ SCRAPING DES COMMENTAIRES ==============
# La fonction de scraping des commentaires  peut prendre assez de temps en raison du nombre de posts et de commentaires à traiter. Nous avons donc fait une sauvegarde intermédiaire tous les 1000 commentaires récupérés.
# Nous avons également limité la profondeur de récupération des commentaires à 2 niveaux pour éviter de récupérer trop de données inutiles.
def scrapper_comments(df, max_comments_per_post=150, save_interval=1000, max_depth=2):
    df_comments = []
    total_count = 0

    try:
        for post_idx, post_id in enumerate(tqdm(df["id"], desc="Téléchargement des commentaires")):
            try:
                submission = reddit.submission(id=post_id)
                submission.comments.replace_more(limit=20)

                # Compteur pour ce post spécifique
                post_comment_count = 0

                # Parcourir les commentaires avec une profondeur limitée
                all_comments = submission.comments.list()

                for comment in all_comments:
                    # Vérification de la profondeur
                    if hasattr(comment, 'depth') and comment.depth <= max_depth:
                        df_comments.append({
                            "post_id": post_id,
                            "comment": comment.body
                        })

                        post_comment_count += 1
                        total_count += 1

                    # Arrêter si on atteint la limite par post qui est de 150 commentaires pour éviter de surcharger le système et avoir un temps de réponse trop long
                    if post_comment_count >= max_comments_per_post:
                        break

                # Sauvegarde à intervalles réguliers
                if total_count >= save_interval:
                    temp_df = pd.DataFrame(df_comments)
                    temp_df.to_csv("/content/df_comments_temp.csv", header=True, index=None)
                    print(f"Sauvegarde intermédiaire: {len(df_comments)} commentaires récupérés")
                    total_count = 0

                # Feedback pour suivre la progression
                if (post_idx + 1) % 10 == 0:
                    print(f"Traité {post_idx + 1}/{len(df)} posts, {len(df_comments)} commentaires au total")

            except Exception as e:
                print(f"Erreur sur le post {post_id}: {str(e)}")
                continue

        # Création du DataFrame final
        final_df = pd.DataFrame(df_comments)
        return final_df

    except Exception as e:
        print(f"Erreur générale: {str(e)}")
        # En cas d'erreur, sauvegarder ce qui a été récupéré jusqu'à présent
        rescue_df = pd.DataFrame(df_comments)
        rescue_df.to_csv("/content/df_comments_rescue.csv", header=True, index=None)
        print(f"Données de secours sauvegardées avec {len(rescue_df)} commentaires")
        return rescue_df

# Exécution
df_comments = scrapper_comments(df=df_posts, max_comments_per_post=150, save_interval=1000, max_depth=2)

# Vérification et sauvegarde du fichier final unique
print(df_comments.head())

# Sauvegarde du fichier final unique avec tous les commentaires
df_comments.to_csv("/content/df_comments.csv", header=True, index=None)

#### Traitement du fichier des commentaires pour avoir une base bien structurée

In [ ]:
# Lire le fichier brut sans parsing strict (utile si structure cassée)
with open("/content/df_comments.csv", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

data = []
current_post_id = None

for line in lines:
    # Si la ligne contient une virgule + commence par un ID
    if ',' in line and not line.startswith('>'):
        parts = line.split(',', 1)
        if len(parts) == 2 and len(parts[0]) < 20:
            current_post_id = parts[0].strip()
            comment = parts[1].strip()
            if comment:  # cas où le commentaire est sur la même ligne
                data.append({"post_id": current_post_id, "comment": comment})
        else:
            pass
    else:
        # C’est une ligne de commentaire rattachée à l’ID précédent
        if current_post_id:
            data.append({"post_id": current_post_id, "comment": line.strip()})

# Création du dataframe propre
df_clean = pd.DataFrame(data)
print(df_clean.head())

# Sauvegarde propre
df_clean.to_csv("/content/df_clean_comments.csv", index=False)
df_clean.head()

### Merging des bases 

In [ ]:
# Merging des posts et des commentaires
df_posts = pd.read_csv("/content/df_posts.csv")
df_comments = pd.read_csv("/content/df_clean_comments.csv")
df_posts = df_posts.rename(columns={"id": "post_id"})
comments_posts_df = df_posts.merge(df_comments, on='post_id', how='left')

# Suppression des lignes vides dans les commentaires
comments_posts_df = comments_posts_df[~comments_posts_df['comment'].isnull()]
comments_posts_df.head()
comments_posts_df.to_csv("/content/df_comments_posts.csv", header = True, index = False)